# Agentic RAG System with LangGraph + Certara.AI

This notebook demonstrates a **Retrieval-Augmented Generation (RAG)** system built on top of **LangGraph** that queries a corpus of policy documents stored in a **Weaviate** vector database, with LLM inference powered by a local **Certara.AI** deployment.

## System Architecture

```
                    ┌─────────────────────────────────────────────────────┐
                    │                  LangGraph Agent                    │
                    │                                                     │
  User ──────►     │  ┌───────────┐    ┌──────────────┐    ┌──────────┐  │
  Question         │  │  Classify  │───►│  Decompose / │───►│ Parallel │  │
                   │  │  Intent    │    │  Rephrase     │    │ Retrieve │  │
                   │  └─────┬─────┘    └──────────────┘    └─────┬────┘  │
                   │        │ need info?                         │        │
                   │        ▼                                    ▼        │
                   │  ┌───────────┐                       ┌──────────┐   │
                   │  │ Clarify / │◄──── User             │ Generate │   │
                   │  │ Ask User  │                       │ Answer   │   │
                   │  └───────────┘                       └─────┬────┘   │
                   │                                            │        │
                   │                                      ┌─────▼────┐   │
                   │                                      │ Evaluate │   │
                   │                                      │ Answer   │   │
                   │                                      └─────┬────┘   │
                   │                                     pass / retry    │
                   └─────────────────────────────────────────────────────┘
                                          │
              ┌───────────────────────────┴───────────────────────────┐
              │                                                       │
     ┌────────▼────────┐                                   ┌─────────▼────────┐
     │  Certara.AI LLM │  (local OpenAI-compatible API)    │  Weaviate Vector │
     │  /v1/chat/...   │                                   │  Database         │
     └─────────────────┘                                   └──────────────────┘
```

## Key Capabilities

| # | Capability | Description |
|---|-----------|-------------|
| 1 | **Clarity Check** | Detects ambiguous or under-specified questions and asks the user for clarification |
| 2 | **Question Decomposition** | Breaks complex questions into sub-queries and runs parallel retrievals |
| 3 | **Answer Evaluation & Retry** | Scores the generated answer and retries retrieval if confidence is low |
| 4 | **Conversational Follow-up** | Handles multi-turn dialogue, deciding when new retrieval is needed |

---
## 1 · Setup & Configuration

In [ ]:
# Install dependencies (run once)
# %pip install langgraph langchain langchain-openai weaviate-client httpx pydantic --quiet

In [ ]:
import os
import json
import httpx
import asyncio
from typing import TypedDict, Annotated, Literal, Optional
from pydantic import BaseModel, Field

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, BaseMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.checkpoint.memory import MemorySaver

In [ ]:
# ── Certara.AI Configuration ──────────────────────────────────────────────────
# These point to your local Certara.AI deployment which exposes an
# OpenAI-compatible API for text generation.

CERTARA_BASE_URL = os.getenv("CERTARA_BASE_URL", "http://localhost:8000/v1")
CERTARA_API_KEY  = os.getenv("CERTARA_API_KEY", "certara-local-key")
CERTARA_MODEL    = os.getenv("CERTARA_MODEL", "certara-default")

# ── Weaviate Configuration ─────────────────────────────────────────────────────
# The vector DB that holds chunked policy documents.

WEAVIATE_URL        = os.getenv("WEAVIATE_URL", "http://localhost:8080")
WEAVIATE_COLLECTION = os.getenv("WEAVIATE_COLLECTION", "PolicyDocuments")

# ── Agent Parameters ──────────────────────────────────────────────────────────
MAX_RETRIEVAL_RETRIES = 2          # Max retry rounds for low-confidence answers
CONFIDENCE_THRESHOLD  = 0.6        # Min confidence to accept an answer
TOP_K_RESULTS         = 5          # Chunks to retrieve per sub-query

print(f"Certara.AI endpoint : {CERTARA_BASE_URL}")
print(f"Weaviate endpoint   : {WEAVIATE_URL}")
print(f"Collection          : {WEAVIATE_COLLECTION}")

---
## 2 · API Client Wrappers

We wrap the three primary interaction modes with Certara.AI into clean helper functions:

| Mode | Endpoint | Purpose |
|------|----------|---------|
| **Generate** | `POST /v1/chat/completions` | LLM text generation |
| **Retrieve** | `POST /v1/retrieve` | Semantic search over Weaviate |
| **RAG (combined)** | `POST /v1/rag` | Retrieve + Generate in one call |

In [ ]:
# ── LLM via LangChain (uses the OpenAI-compatible /v1/chat/completions) ──────

llm = ChatOpenAI(
    base_url=CERTARA_BASE_URL,
    api_key=CERTARA_API_KEY,
    model=CERTARA_MODEL,
    temperature=0.1,
)

# Quick smoke test (will fail gracefully if Certara.AI is not running)
try:
    resp = llm.invoke([HumanMessage(content="Say hello in one sentence.")])
    print(f"LLM connected ✓  Response: {resp.content[:80]}")
except Exception as e:
    print(f"LLM not reachable (expected if Certara.AI is not running): {e}")

In [ ]:
# ── Weaviate Retrieval Client ─────────────────────────────────────────────────

async def retrieve_from_weaviate(
    query: str,
    top_k: int = TOP_K_RESULTS,
    collection: str = WEAVIATE_COLLECTION,
) -> list[dict]:
    """
    Mode 2: Retrieve relevant chunks from the Weaviate vector database
    via the Certara.AI retrieval endpoint.

    Returns a list of dicts with keys: text, source, score, metadata.
    """
    payload = {
        "query": query,
        "collection": collection,
        "top_k": top_k,
    }
    async with httpx.AsyncClient(timeout=30) as client:
        response = await client.post(
            f"{CERTARA_BASE_URL.rstrip('/v1')}/v1/retrieve",
            json=payload,
            headers={"Authorization": f"Bearer {CERTARA_API_KEY}"},
        )
        response.raise_for_status()
        data = response.json()

    # Normalize the response into a consistent format
    results = []
    for item in data.get("results", []):
        results.append({
            "text": item.get("text", item.get("content", "")),
            "source": item.get("source", item.get("metadata", {}).get("source", "unknown")),
            "score": item.get("score", item.get("distance", 0.0)),
            "metadata": item.get("metadata", {}),
        })
    return results


async def parallel_retrieve(queries: list[str], top_k: int = TOP_K_RESULTS) -> list[dict]:
    """
    Run multiple retrieval queries in parallel and deduplicate results.
    """
    tasks = [retrieve_from_weaviate(q, top_k=top_k) for q in queries]
    all_results = await asyncio.gather(*tasks, return_exceptions=True)

    # Flatten and deduplicate by text content
    seen_texts = set()
    deduplicated = []
    for result_set in all_results:
        if isinstance(result_set, Exception):
            print(f"  ⚠ Retrieval error: {result_set}")
            continue
        for r in result_set:
            text_key = r["text"][:200]  # dedup on first 200 chars
            if text_key not in seen_texts:
                seen_texts.add(text_key)
                deduplicated.append(r)

    # Sort by relevance score (higher is better)
    deduplicated.sort(key=lambda x: x["score"], reverse=True)
    return deduplicated


print("Retrieval functions defined ✓")

In [ ]:
# ── Combined RAG Client ───────────────────────────────────────────────────────

async def certara_rag_call(
    query: str,
    top_k: int = TOP_K_RESULTS,
    system_prompt: str = "You are a helpful policy assistant.",
) -> dict:
    """
    Mode 3: Combined Retrieve + Generate call to Certara.AI.
    The server handles both retrieval and generation in a single request.

    Returns: {"answer": str, "sources": list[dict]}
    """
    payload = {
        "query": query,
        "collection": WEAVIATE_COLLECTION,
        "top_k": top_k,
        "system_prompt": system_prompt,
        "model": CERTARA_MODEL,
    }
    async with httpx.AsyncClient(timeout=60) as client:
        response = await client.post(
            f"{CERTARA_BASE_URL.rstrip('/v1')}/v1/rag",
            json=payload,
            headers={"Authorization": f"Bearer {CERTARA_API_KEY}"},
        )
        response.raise_for_status()
        return response.json()


print("RAG client defined ✓")

---
## 3 · Graph State Definition

LangGraph uses a typed state object that flows through every node in the graph. Our state tracks the full conversation history, retrieved sources, decomposed sub-queries, confidence scores, and retry counts.

In [ ]:
class RAGState(TypedDict):
    """State that flows through the LangGraph RAG pipeline."""

    # ── Conversation ──────────────────────────────────────────────────────────
    messages: Annotated[list[BaseMessage], add_messages]  # full chat history
    current_query: str                                     # latest user question

    # ── Intent Classification ─────────────────────────────────────────────────
    intent: str              # "clear_question" | "needs_clarification" | "follow_up" | "chitchat"
    needs_retrieval: bool    # whether follow-up requires new retrieval

    # ── Decomposition & Retrieval ─────────────────────────────────────────────
    sub_queries: list[str]           # decomposed / rephrased queries
    retrieved_sources: list[dict]    # chunks from Weaviate

    # ── Generation & Evaluation ───────────────────────────────────────────────
    draft_answer: str        # generated answer (before evaluation)
    confidence: float        # 0.0 – 1.0 self-assessed confidence
    retry_count: int         # number of retrieval retries so far
    final_answer: str        # accepted answer to present to user


print("State schema defined ✓")

---
## 4 · Graph Nodes

Each node is an async function that reads state, performs work, and returns a partial state update.

### 4.1 — Classify Intent

The first node examines the user's message in context of the conversation history and classifies it as:
- **`clear_question`** — well-formed policy question → proceed to decomposition  
- **`needs_clarification`** — ambiguous or missing information → ask user  
- **`follow_up`** — continuation of a prior topic → check if new retrieval is needed  
- **`chitchat`** — greeting or off-topic → respond directly

In [ ]:
CLASSIFY_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are an intent classifier for a policy Q&A system.

Given the conversation history and the latest user message, classify the intent
into EXACTLY ONE of the following categories and respond with valid JSON only:

{{
  "intent": "clear_question" | "needs_clarification" | "follow_up" | "chitchat",
  "needs_retrieval": true | false,
  "reasoning": "<one sentence explaining your classification>"
}}

Guidelines:
- "clear_question": The user asks a specific, answerable question about policy.
- "needs_clarification": The question is too vague, ambiguous, or references
  something unspecified (e.g., "What's the policy?" without saying WHICH policy,
  or "Tell me about the rules" without context).
- "follow_up": The user is continuing a previous topic — asking for more detail,
  rephrasing, or adding a related question. Set needs_retrieval=true if answering
  likely requires new information beyond what was already retrieved.
- "chitchat": Greetings, thanks, or off-topic messages.

Respond ONLY with the JSON object, no other text."""),
    ("human", """Conversation so far:
{history}

Latest user message: {query}"""),
])


async def classify_intent(state: RAGState) -> dict:
    """Node: Classify the user's intent."""
    # Build a readable history string
    history_lines = []
    for msg in state.get("messages", [])[:-1]:  # exclude latest
        role = "User" if isinstance(msg, HumanMessage) else "Assistant"
        history_lines.append(f"{role}: {msg.content}")
    history_str = "\n".join(history_lines[-10:]) or "(no prior conversation)"

    chain = CLASSIFY_PROMPT | llm | StrOutputParser()
    raw = await chain.ainvoke({"history": history_str, "query": state["current_query"]})

    # Parse the JSON response
    try:
        parsed = json.loads(raw)
    except json.JSONDecodeError:
        # Fallback if the LLM didn't return clean JSON
        parsed = {"intent": "clear_question", "needs_retrieval": True, "reasoning": "Parse error, defaulting."}

    print(f"  🏷  Intent: {parsed['intent']}  |  Needs retrieval: {parsed.get('needs_retrieval', True)}")
    print(f"     Reasoning: {parsed.get('reasoning', '')}")

    return {
        "intent": parsed["intent"],
        "needs_retrieval": parsed.get("needs_retrieval", True),
    }


print("classify_intent node defined ✓")

### 4.2 — Clarify (Ask User for More Information)

When the question is too vague, this node generates a clarification request.

In [ ]:
CLARIFY_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful policy assistant. The user's question is
ambiguous or lacks sufficient detail for you to look up an answer in the policy
database. Write a concise, friendly message asking the user to clarify.
Be specific about WHAT information is missing."""),
    ("human", "{query}"),
])


async def clarify(state: RAGState) -> dict:
    """Node: Ask the user for clarification."""
    chain = CLARIFY_PROMPT | llm | StrOutputParser()
    clarification = await chain.ainvoke({"query": state["current_query"]})

    print(f"  ❓ Asking for clarification")

    return {
        "final_answer": clarification,
        "messages": [AIMessage(content=clarification)],
    }


print("clarify node defined ✓")

### 4.3 — Decompose & Rephrase Queries

Complex questions are broken into focused sub-queries that are individually optimized for vector-similarity retrieval. This improves recall significantly over sending a single long question.

In [ ]:
DECOMPOSE_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are a query decomposition engine for a policy document search system.

Given the user's question (and conversation history for context), produce 1–4
focused search queries that together cover the full information need.

Rules:
- Each sub-query should be a short, specific phrase optimized for semantic search.
- Replace pronouns with their referents using conversation context.
- If the question is already simple and specific, return it as a single query.
- Do NOT add queries for information the user did not ask about.

Respond with valid JSON only:
{{"queries": ["query 1", "query 2", ...]}}"""),
    ("human", """Conversation context:
{history}

Current question: {query}"""),
])


async def decompose_query(state: RAGState) -> dict:
    """Node: Decompose / rephrase the question into retrieval-optimized sub-queries."""
    history_lines = []
    for msg in state.get("messages", [])[:-1]:
        role = "User" if isinstance(msg, HumanMessage) else "Assistant"
        history_lines.append(f"{role}: {msg.content}")
    history_str = "\n".join(history_lines[-6:]) or "(none)"

    chain = DECOMPOSE_PROMPT | llm | StrOutputParser()
    raw = await chain.ainvoke({"history": history_str, "query": state["current_query"]})

    try:
        parsed = json.loads(raw)
        sub_queries = parsed.get("queries", [state["current_query"]])
    except json.JSONDecodeError:
        sub_queries = [state["current_query"]]

    print(f"  🔀 Decomposed into {len(sub_queries)} sub-queries:")
    for i, q in enumerate(sub_queries, 1):
        print(f"     {i}. {q}")

    return {"sub_queries": sub_queries}


print("decompose_query node defined ✓")

### 4.4 — Parallel Retrieval

All sub-queries are dispatched to Weaviate **in parallel** via `asyncio.gather`. Results are deduplicated and sorted by relevance score.

In [ ]:
async def retrieve(state: RAGState) -> dict:
    """Node: Run parallel retrieval for all sub-queries against Weaviate."""
    queries = state.get("sub_queries", [state["current_query"]])

    print(f"  🔍 Retrieving from Weaviate ({len(queries)} parallel queries)...")
    sources = await parallel_retrieve(queries, top_k=TOP_K_RESULTS)

    # Merge with any previously retrieved sources (for retries)
    existing = state.get("retrieved_sources", [])
    existing_texts = {s["text"][:200] for s in existing}
    for s in sources:
        if s["text"][:200] not in existing_texts:
            existing.append(s)

    print(f"  📄 Retrieved {len(sources)} new chunks ({len(existing)} total)")

    return {"retrieved_sources": existing}


print("retrieve node defined ✓")

### 4.5 — Generate Answer

The generation node assembles retrieved context into a prompt and calls the LLM to produce a grounded answer with citations.

In [ ]:
GENERATE_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are a policy expert assistant. Answer the user's question
using ONLY the provided source documents. Follow these rules:

1. Base your answer strictly on the retrieved sources.
2. Cite sources using [Source: <filename>] notation.
3. If the sources do not contain enough information, say so clearly.
4. Be concise but thorough.
5. If multiple sources conflict, note the discrepancy.

Retrieved Sources:
{context}"""),
    ("human", "{query}"),
])


async def generate_answer(state: RAGState) -> dict:
    """Node: Generate an answer grounded in retrieved sources."""
    sources = state.get("retrieved_sources", [])

    # Format context block
    context_parts = []
    for i, s in enumerate(sources[:10], 1):  # Limit to top 10
        context_parts.append(
            f"--- Source {i} [Source: {s['source']}] (score: {s['score']:.3f}) ---\n{s['text']}"
        )
    context_str = "\n\n".join(context_parts) or "(No sources retrieved)"

    chain = GENERATE_PROMPT | llm | StrOutputParser()
    answer = await chain.ainvoke({"context": context_str, "query": state["current_query"]})

    print(f"  ✍  Draft answer generated ({len(answer)} chars)")

    return {"draft_answer": answer}


print("generate_answer node defined ✓")

### 4.6 — Evaluate Answer

The evaluation node uses a separate LLM call to score the draft answer for:
- **Relevance**: Does it address what was asked?
- **Groundedness**: Is it supported by the retrieved sources?
- **Completeness**: Does it fully answer the question?

If confidence is below the threshold, the system retries with refined queries.

In [ ]:
EVALUATE_PROMPT = ChatPromptTemplate.from_messages([
    ("system", """You are a critical evaluator for a RAG system. Assess the quality
of the generated answer given the original question and the retrieved sources.

Score each dimension from 0.0 to 1.0:
- relevance: Does the answer address the question?
- groundedness: Is every claim supported by the sources?
- completeness: Does the answer fully address all parts of the question?

Also suggest what additional information to search for if the answer is incomplete.

Respond with valid JSON only:
{{
  "relevance": 0.0-1.0,
  "groundedness": 0.0-1.0,
  "completeness": 0.0-1.0,
  "overall_confidence": 0.0-1.0,
  "missing_info": "description of what's missing, or 'none'",
  "retry_queries": ["suggested search query 1", ...]
}}"""),
    ("human", """Question: {query}

Generated Answer:
{answer}

Sources used (count): {source_count}"""),
])


async def evaluate_answer(state: RAGState) -> dict:
    """Node: Evaluate the draft answer and decide whether to accept or retry."""
    chain = EVALUATE_PROMPT | llm | StrOutputParser()
    raw = await chain.ainvoke({
        "query": state["current_query"],
        "answer": state["draft_answer"],
        "source_count": len(state.get("retrieved_sources", [])),
    })

    try:
        evaluation = json.loads(raw)
    except json.JSONDecodeError:
        evaluation = {"overall_confidence": 0.7, "retry_queries": []}

    confidence = evaluation.get("overall_confidence", 0.7)
    retry_queries = evaluation.get("retry_queries", [])

    print(f"  📊 Evaluation — Confidence: {confidence:.2f}")
    print(f"     Relevance:    {evaluation.get('relevance', 'N/A')}")
    print(f"     Groundedness: {evaluation.get('groundedness', 'N/A')}")
    print(f"     Completeness: {evaluation.get('completeness', 'N/A')}")
    if evaluation.get("missing_info", "none") != "none":
        print(f"     Missing info: {evaluation['missing_info']}")

    return {
        "confidence": confidence,
        "sub_queries": retry_queries if confidence < CONFIDENCE_THRESHOLD else [],
        "retry_count": state.get("retry_count", 0) + (1 if confidence < CONFIDENCE_THRESHOLD else 0),
    }


print("evaluate_answer node defined ✓")

### 4.7 — Finalize Answer

Once the answer passes evaluation (or max retries are exhausted), this node packages the final response.

In [ ]:
async def finalize_answer(state: RAGState) -> dict:
    """Node: Accept the draft answer as final and add it to the message history."""
    answer = state["draft_answer"]

    # If confidence was low and retries exhausted, prepend a caveat
    if state.get("confidence", 1.0) < CONFIDENCE_THRESHOLD:
        caveat = (
            "⚠️ **Note:** I was unable to find highly relevant sources for all "
            "parts of your question. The answer below is based on the best "
            "available information, but may be incomplete.\n\n"
        )
        answer = caveat + answer

    # Append source citations summary
    sources = state.get("retrieved_sources", [])
    if sources:
        unique_sources = sorted(set(s["source"] for s in sources))
        source_list = "\n".join(f"  • {s}" for s in unique_sources)
        answer += f"\n\n---\n**Sources consulted:**\n{source_list}"

    print(f"  ✅ Answer finalized")

    return {
        "final_answer": answer,
        "messages": [AIMessage(content=answer)],
    }


print("finalize_answer node defined ✓")

### 4.8 — Handle Chitchat

Simple conversational responses that don't require retrieval.

In [ ]:
async def handle_chitchat(state: RAGState) -> dict:
    """Node: Respond to non-policy chitchat (greetings, thanks, etc.)."""
    response = await llm.ainvoke([
        SystemMessage(content=(
            "You are a friendly policy assistant. The user sent a casual message "
            "(greeting, thanks, etc.). Respond briefly and warmly, and let them "
            "know you're here to help with policy questions."
        )),
        HumanMessage(content=state["current_query"]),
    ])

    print(f"  💬 Chitchat response")

    return {
        "final_answer": response.content,
        "messages": [AIMessage(content=response.content)],
    }


print("handle_chitchat node defined ✓")

---
## 5 · Routing Logic (Conditional Edges)

LangGraph uses conditional edges to decide which node to visit next. These functions implement our routing decisions.

In [ ]:
def route_after_classification(state: RAGState) -> str:
    """
    After classifying intent, decide the next step:
      - needs_clarification → ask user
      - chitchat → respond directly
      - follow_up (no retrieval needed) → generate from existing sources
      - clear_question / follow_up (retrieval needed) → decompose & retrieve
    """
    intent = state.get("intent", "clear_question")

    if intent == "needs_clarification":
        return "clarify"
    elif intent == "chitchat":
        return "handle_chitchat"
    elif intent == "follow_up" and not state.get("needs_retrieval", True):
        # Follow-up that can be answered from existing context
        return "generate_answer"
    else:
        # clear_question or follow_up needing retrieval
        return "decompose_query"


def route_after_evaluation(state: RAGState) -> str:
    """
    After evaluating the answer, decide whether to:
      - Accept the answer (confidence >= threshold or retries exhausted)
      - Retry with new queries (confidence < threshold and retries remaining)
    """
    confidence = state.get("confidence", 1.0)
    retry_count = state.get("retry_count", 0)

    if confidence >= CONFIDENCE_THRESHOLD:
        print(f"  ✅ Confidence {confidence:.2f} >= {CONFIDENCE_THRESHOLD} → accepting")
        return "finalize_answer"
    elif retry_count >= MAX_RETRIEVAL_RETRIES:
        print(f"  ⚠️  Max retries ({MAX_RETRIEVAL_RETRIES}) reached → accepting with caveat")
        return "finalize_answer"
    else:
        print(f"  🔄 Confidence {confidence:.2f} < {CONFIDENCE_THRESHOLD} → retrying (attempt {retry_count})")
        return "retrieve"  # retry with the new sub_queries set by evaluate


print("Routing functions defined ✓")

---
## 6 · Build the LangGraph

Now we assemble all nodes and edges into the complete graph.

In [ ]:
def build_rag_graph() -> StateGraph:
    """Construct the full RAG agent graph."""

    graph = StateGraph(RAGState)

    # ── Add nodes ─────────────────────────────────────────────────────────────
    graph.add_node("classify_intent",  classify_intent)
    graph.add_node("clarify",          clarify)
    graph.add_node("handle_chitchat",  handle_chitchat)
    graph.add_node("decompose_query",  decompose_query)
    graph.add_node("retrieve",         retrieve)
    graph.add_node("generate_answer",  generate_answer)
    graph.add_node("evaluate_answer",  evaluate_answer)
    graph.add_node("finalize_answer",  finalize_answer)

    # ── Entry point ───────────────────────────────────────────────────────────
    graph.set_entry_point("classify_intent")

    # ── Conditional: after classification ─────────────────────────────────────
    graph.add_conditional_edges(
        "classify_intent",
        route_after_classification,
        {
            "clarify":          "clarify",
            "handle_chitchat":  "handle_chitchat",
            "decompose_query":  "decompose_query",
            "generate_answer":  "generate_answer",
        },
    )

    # ── Linear edges ──────────────────────────────────────────────────────────
    graph.add_edge("decompose_query", "retrieve")
    graph.add_edge("retrieve",        "generate_answer")
    graph.add_edge("generate_answer", "evaluate_answer")

    # ── Conditional: after evaluation (accept vs retry) ───────────────────────
    graph.add_conditional_edges(
        "evaluate_answer",
        route_after_evaluation,
        {
            "finalize_answer": "finalize_answer",
            "retrieve":        "retrieve",   # retry loop
        },
    )

    # ── Terminal edges ─────────────────────────────────────────────────────────
    graph.add_edge("clarify",         END)
    graph.add_edge("handle_chitchat", END)
    graph.add_edge("finalize_answer", END)

    return graph


# Build and compile with memory for multi-turn conversations
memory = MemorySaver()
rag_graph = build_rag_graph()
rag_app = rag_graph.compile(checkpointer=memory)

print("Graph compiled ✓")

### 6.1 — Visualize the Graph

LangGraph can render a visual representation of the state machine.

In [ ]:
# Display the graph structure (requires graphviz or mermaid support)
try:
    from IPython.display import Image, display
    img_bytes = rag_app.get_graph().draw_mermaid_png()
    display(Image(img_bytes))
except Exception as e:
    # Fallback: print the mermaid diagram as text
    print("Graph visualization (Mermaid format):")
    print(rag_app.get_graph().draw_mermaid())

---
## 7 · Conversation Runner

A helper function that manages the interaction loop — feeding user messages into the graph, displaying results, and maintaining session state across turns.

In [ ]:
async def ask(
    question: str,
    thread_id: str = "default-session",
    verbose: bool = True,
) -> str:
    """
    Send a question to the RAG agent and return the response.

    Args:
        question: The user's question or message.
        thread_id: Session ID for multi-turn conversation tracking.
        verbose: If True, print step-by-step progress.

    Returns:
        The agent's final response string.
    """
    if verbose:
        print(f"\n{'='*70}")
        print(f"🧑 User: {question}")
        print(f"{'='*70}")

    config = {"configurable": {"thread_id": thread_id}}

    input_state = {
        "current_query": question,
        "messages": [HumanMessage(content=question)],
        "retry_count": 0,
        "retrieved_sources": [],
    }

    # Stream through the graph for visibility into each step
    final_state = None
    async for event in rag_app.astream(input_state, config=config):
        for node_name, node_output in event.items():
            if verbose:
                print(f"\n  ── Node: {node_name} ──")
            final_state = node_output

    answer = final_state.get("final_answer", "I wasn't able to generate a response.")

    if verbose:
        print(f"\n{'─'*70}")
        print(f"🤖 Assistant:\n{answer}")
        print(f"{'─'*70}\n")

    return answer


print("ask() helper defined ✓")

---
## 8 · Example Conversations

The following cells demonstrate each of the four key capabilities. **These will only produce real outputs when Certara.AI and Weaviate are running locally.** Otherwise, they serve as runnable templates showing the intended flow.

### 8.1 — Capability 1: Clarity Check & Clarification Request

The system should detect that the question is too vague and ask for specifics.

In [ ]:
# Example: Vague question that should trigger clarification
await ask(
    "What's the policy on that?",
    thread_id="demo-clarity",
)

# Expected flow: classify_intent → "needs_clarification" → clarify → END

In [ ]:
# Another ambiguous example
await ask(
    "Tell me about the rules.",
    thread_id="demo-clarity-2",
)

# Expected: System asks "Which rules are you referring to? For example,
# are you asking about travel policy, expense reimbursement, data privacy...?"

### 8.2 — Capability 2: Question Decomposition & Parallel Retrieval

A complex question is split into focused sub-queries that each retrieve relevant chunks independently.

In [ ]:
# Example: Complex multi-part question
await ask(
    "What is the maximum reimbursement for international travel, "
    "and does the per-diem rate differ between Europe and Asia?",
    thread_id="demo-decompose",
)

# Expected decomposition:
#   1. "maximum reimbursement international travel policy"
#   2. "per-diem rate Europe travel policy"
#   3. "per-diem rate Asia travel policy"
# All three queries run in parallel against Weaviate.

### 8.3 — Capability 3: Answer Evaluation & Retry

If the initial retrieval doesn't yield a confident answer, the system automatically retries with refined queries.

In [ ]:
# Example: Niche question that may require retry
await ask(
    "Under what circumstances can an employee request an exception to the "
    "standard equipment procurement policy for specialized research hardware?",
    thread_id="demo-retry",
)

# Expected flow:
#   classify → decompose → retrieve → generate → evaluate
#   If confidence < 0.6: → retrieve (with new queries) → generate → evaluate
#   Repeat up to MAX_RETRIEVAL_RETRIES times, then finalize

### 8.4 — Capability 4: Multi-turn Follow-up Handling

The system tracks conversation context across turns and decides when follow-up questions need new retrieval vs. when they can be answered from existing context.

In [ ]:
SESSION = "demo-followup"

# Turn 1: Initial question
await ask(
    "What is our remote work policy?",
    thread_id=SESSION,
)

In [ ]:
# Turn 2: Follow-up that can be answered from existing context
await ask(
    "Can you summarize that in bullet points?",
    thread_id=SESSION,
)

# Expected: intent="follow_up", needs_retrieval=False
# → skips retrieval, reformats existing answer

In [ ]:
# Turn 3: Follow-up that DOES need new retrieval
await ask(
    "How does this differ from the policy for contractors?",
    thread_id=SESSION,
)

# Expected: intent="follow_up", needs_retrieval=True
# → decomposes into queries about contractor remote work policy
# → retrieves new sources and generates comparative answer

In [ ]:
# Turn 4: Chitchat
await ask(
    "Thanks, that's really helpful!",
    thread_id=SESSION,
)

# Expected: intent="chitchat" → friendly response, no retrieval

---
## 9 · Interactive Chat Loop

Run this cell for an interactive session. Type `quit` or `exit` to stop.

In [ ]:
async def interactive_chat():
    """Run an interactive chat loop with the RAG agent."""
    session_id = "interactive-session"
    print("╔══════════════════════════════════════════════════════════════════╗")
    print("║       Policy RAG Assistant — Interactive Mode                   ║")
    print("║  Ask questions about company policies. Type 'quit' to exit.     ║")
    print("╚══════════════════════════════════════════════════════════════════╝\n")

    while True:
        try:
            user_input = input("You: ").strip()
        except (EOFError, KeyboardInterrupt):
            break

        if not user_input:
            continue
        if user_input.lower() in ("quit", "exit", "q"):
            print("\nGoodbye!")
            break

        await ask(user_input, thread_id=session_id)


# Uncomment to run:
# await interactive_chat()

---
## 10 · Debugging & Inspection Utilities

Helpers for inspecting graph state, tracing execution, and testing individual components.

In [ ]:
def inspect_thread_state(thread_id: str = "default-session"):
    """
    Print the current state snapshot for a conversation thread.
    Useful for debugging multi-turn interactions.
    """
    config = {"configurable": {"thread_id": thread_id}}
    snapshot = rag_app.get_state(config)

    if not snapshot or not snapshot.values:
        print(f"No state found for thread '{thread_id}'")
        return

    state = snapshot.values
    print(f"\n📋 State for thread '{thread_id}'")
    print(f"   Messages:          {len(state.get('messages', []))}")
    print(f"   Current query:     {state.get('current_query', 'N/A')}")
    print(f"   Intent:            {state.get('intent', 'N/A')}")
    print(f"   Sub-queries:       {state.get('sub_queries', [])}")
    print(f"   Sources retrieved: {len(state.get('retrieved_sources', []))}")
    print(f"   Confidence:        {state.get('confidence', 'N/A')}")
    print(f"   Retry count:       {state.get('retry_count', 0)}")

    # Print message history
    print(f"\n   Message History:")
    for i, msg in enumerate(state.get("messages", [])):
        role = "🧑 User" if isinstance(msg, HumanMessage) else "🤖 Asst"
        content_preview = msg.content[:100] + ("..." if len(msg.content) > 100 else "")
        print(f"   [{i}] {role}: {content_preview}")


# Example usage:
# inspect_thread_state("demo-followup")

In [ ]:
async def test_retrieval(query: str):
    """
    Test retrieval independently — useful for debugging search quality
    without running the full pipeline.
    """
    print(f"Testing retrieval for: '{query}'")
    print(f"{'─'*50}")

    try:
        results = await retrieve_from_weaviate(query)
        for i, r in enumerate(results, 1):
            print(f"\n  [{i}] Score: {r['score']:.4f}  |  Source: {r['source']}")
            print(f"      {r['text'][:150]}...")
        print(f"\nTotal results: {len(results)}")
    except Exception as e:
        print(f"Retrieval failed: {e}")


# Example:
# await test_retrieval("remote work eligibility requirements")

In [ ]:
async def test_combined_rag(query: str):
    """
    Test the combined RAG endpoint independently.
    """
    print(f"Testing combined RAG for: '{query}'")
    print(f"{'─'*50}")

    try:
        result = await certara_rag_call(query)
        print(f"\nAnswer:\n{result.get('answer', 'N/A')}")
        print(f"\nSources:")
        for s in result.get("sources", []):
            print(f"  • {s.get('source', 'unknown')} (score: {s.get('score', 'N/A')})")
    except Exception as e:
        print(f"RAG call failed: {e}")


# Example:
# await test_combined_rag("What is the vacation accrual policy?")

---
## 11 · Configuration Notes & Customization

### Adapting to Your Environment

| Setting | Where to Change | Notes |
|---------|----------------|-------|
| Certara.AI URL | `CERTARA_BASE_URL` in §1 | Must expose OpenAI-compatible `/v1/chat/completions` |
| Model name | `CERTARA_MODEL` in §1 | Match your deployed model identifier |
| Weaviate URL | `WEAVIATE_URL` in §1 | Default Weaviate port is 8080 |
| Collection name | `WEAVIATE_COLLECTION` in §1 | Must match your ingested document collection |
| Confidence threshold | `CONFIDENCE_THRESHOLD` in §1 | Lower = more permissive, Higher = more retries |
| Max retries | `MAX_RETRIEVAL_RETRIES` in §1 | Cap on retrieval retry loops |
| Chunks per query | `TOP_K_RESULTS` in §1 | More chunks = more context but higher latency |

### API Endpoint Assumptions

This notebook assumes your Certara.AI deployment exposes three endpoints:

```
POST /v1/chat/completions    # Standard OpenAI-compatible generation
POST /v1/retrieve            # Vector search against Weaviate
POST /v1/rag                 # Combined retrieve + generate
```

If your API differs, update the `retrieve_from_weaviate()` and `certara_rag_call()` functions in §2 to match your endpoint signatures and response formats.

### Extending the Graph

Common extensions you might add:

- **Guardrails node** — check generated answers for policy compliance or PII before returning
- **Routing to specialized sub-graphs** — e.g., different retrieval strategies for HR vs. Finance policies
- **Human-in-the-loop** — use LangGraph's `interrupt_before` to pause before certain nodes and require approval
- **Streaming** — replace `astream` with `astream_events` for token-level streaming to the UI